#Collect block data from etherchain API

In [4]:
# import modules
from pymongo import MongoClient
import pandas as pd
import requests
import time

In [5]:
# connect to the hosted MongoDB instance
client = MongoClient()
db = client['etherchain']
collection = db['blocks']
# the transactions with be uniquely identfied by their unique blockid, preventing duplicates
collection.create_index([('number', 1)], unique=True)

u'number_1'

In [6]:
def collect_blocks(count, offset):
    """
    DESCRIPTION:
        Collects block data from etherchain.org API
        https://etherchain.org/documentation/api/
    
    INPUT:
        - offset: the number of block ids to skip
        - count: the number of blocks to return (max 100 blocks per request)
        - sleeptime: number of seconds to sleep between api requests
    OUTPUT:
        - stores each blockchain block as a document in a MongoDB collection
    """
    
    BASE_URL = 'https://etherchain.org/api/blocks/{}/{}'.format(offset, count)
    r = requests.get(BASE_URL)

    if r.status_code != 200:
        print('status code = {}'.format(r.status_code))

    else:
    # store each block in our mongo database
        r = convert_price(r.json())

        for row in range(len(r['data'])):
            try:

                collection.insert_one(r['data'][row])

            except Exception as e:
                print e
                #print(r['data'][row])
                #pass
        
def convert_price(json_dict):
    
    for tx in json_dict['data']:
        tx['size'] = float(tx['size'])
        tx['tx_count'] = float(tx['tx_count'])
        tx['uncle_count'] = float(tx['uncle_count'])
        tx['reward'] = float(tx['reward'])
        tx['totalFee'] = float(tx['totalFee'])
        tx['difficulty'] = float(tx['difficulty'])
    
        return json_dict

def call_api_blocks():
    
    offset = 1253000
    while True:
        collect_blocks(100, offset)
        offset += 200
        if offset % 1000 == 0:
            print('offset: {}'.format(offset))
        time.sleep(20)

In [13]:
call_api_blocks()

E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059637 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059636 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059635 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059634 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059633 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059632 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059631 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059630 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059629 }
E11000 duplicate key error collection: etherchain.blocks_2 index: number_1 dup key: { : 3059628 }
E11000 duplicate key

ConnectionError: HTTPSConnectionPool(host='etherchain.org', port=443): Max retries exceeded with url: /api/blocks/1503000/100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10e7f5190>: Failed to establish a new connection: [Errno 65] No route to host',))